## Quarterly billing or new project report 
* Pull a quarterly billing or new project report from the TIRCP spreadsheet of any new allocations (new project IDs) made since the previous report?
* New projects highlighted in yellow
* Sort small Project ID to larger project ID
* Include existing projects (no highlight)
* Each cycle its own tab!

### Columns
* Project ID	
* EA	
* Ph.	
* Dist.	
* Recipient	
* Project	
* Amount Available	
* Allocation Amount	
* Fund Type	
* Budget Year	
* Appropriation	
* CTC Allocation Date

In [53]:
import A1_data_prep
import A2_tableau
import A7_accounting_analysis
import numpy as np
import pandas as pd
from babel.numbers import format_currency
from calitp import *

pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [54]:
# Load in sheets
project = A1_data_prep.clean_project()
alloc = A1_data_prep.clean_allocation()

/opt/conda/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


In [55]:
# project.columns

In [56]:
# Columns for allocation subset
alloc_subset = [
    "allocation_award_year",
    "allocation_ppno",
    "allocation_project_id",
    "allocation_ea",
    "allocation_grant_recipient",
    "allocation_phase",
    "allocation_allocation_amount",
    "allocation_sb1_funding",
    "allocation_sb1_budget_year",
    "allocation_ggrf_funding",
    "allocation_ggrf_budget_year",
    "allocation_allocation_date",
]

In [57]:
alloc2 = alloc[alloc_subset]

In [58]:
alloc2 = A7_accounting_analysis.clean_project_ids(
    alloc2,
    "allocation_project_id",
)

/home/jovyan/data-analyses/tircp/A7_accounting_analysis.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[project_id_col] = df[project_id_col].astype("str")
/home/jovyan/data-analyses/tircp/A7_accounting_analysis.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[project_id_col] = df[project_id_col].apply(
/home/jovyan/data-analyses/tircp/A7_accounting_analysis.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer]

In [59]:
alloc2 = (alloc2.loc[alloc2.allocation_project_id != "None"]).reset_index(drop=True)

In [60]:
len(alloc2), len(alloc)

(276, 370)

In [61]:
alloc2.allocation_project_id = alloc2.allocation_project_id.apply(
    pd.to_numeric, errors="coerce"
)

In [62]:
project_subset = [
    "project_grant_recipient",
    "project_project_title",
    "project_tircp_award_amount__$_",
    "project_ppno",
    "project_district",
    "project_award_year",
]

In [63]:
# Subset
project2 = project[project_subset]

In [64]:
# Filter
project2 = (project2.loc[project2.project_award_year == 2016]).reset_index(drop=True)

In [65]:
# Merge the 2
m1 = pd.merge(
    alloc2,
    project2,
    how="inner",
    left_on=["allocation_ppno", "allocation_award_year"],
    right_on=["project_ppno", "project_award_year"],
    indicator=True,
)

In [66]:
m1.shape

(34, 19)

In [67]:
# Melt based on project id
ggrf_sb1_values = pd.melt(
    m1,
    id_vars=["allocation_project_id"],
    value_vars=["allocation_sb1_funding", "allocation_ggrf_funding"],
)

In [68]:
# Keep only values above 1
ggrf_sb1_values = (
    (ggrf_sb1_values.loc[ggrf_sb1_values["value"] > 0.00])
    .reset_index(drop=True)
    .rename(columns={"variable": "Fund Type", "value": "Allocation Amount"})
)

In [78]:
# Merge the m1_t with original m1
m2 = pd.merge(
    m1.drop(columns=["_merge"]),
    ggrf_sb1_values,
    how="left",
    on=["allocation_project_id"],
)

In [79]:
m3 = m2.drop_duplicates().sort_values("allocation_project_id")

In [80]:
m3.shape

(36, 20)

In [81]:
m3 = m2.drop_duplicates().sort_values("allocation_project_id")

In [94]:
groupby_cols = [
    "project_project_title",
    "allocation_grant_recipient",
    "project_district",
    "project_tircp_award_amount__$_",
    "allocation_phase",
    "allocation_project_id",
    "allocation_ea",
    "allocation_sb1_budget_year",
    "allocation_ggrf_budget_year",
    "allocation_award_year",
        "Fund Type",
]

In [95]:
m3.groupby(groupby_cols).agg({"Allocation Amount": "max"})

Allocation Amount
project_project_title                                                                                allocation_grant_recipient                                 project_district project_tircp_award_amount__$_ allocation_phase allocation_project_id allocation_ea allocation_sb1_budget_year allocation_ggrf_budget_year Fund Type               allocation_award_year                   
 ACE Near-Term Capacity Improvement Program\n                                                        San Joaquin Regional Rail Commission                       VAR              16459000.00                    CONST            18000010              R368GB        None                       2016-17                     allocation_ggrf_funding 2016.00                       7500000.00
                                                                                                                                                                                                                                 22000242              R368GD        2020-21                    None                        allocation_sb1_funding  2016.00                       8459000.00
                                                                                                                                                                                                                PA&ED            18000009              R368GA        None                       2016-17                     allocation_ggrf_funding 2016.00                        250000.00
                                                                                                                                                                                                                PS&E             18000287              R368GC        None                       2016-17                     allocation_ggrf_funding 2016.00                        500000.00
Airport Metro Connector 96th Street Station/Metro Green Line Extension to LAX                        Los Angeles County Metropolitan Transportation Authority   7                40000000.00                    CONST            20000197              R435GA        None                       2019-20                     allocation_ggrf_funding 2016.00                      40000000.00
All Aboard: Transforming Southern California Rail Travel                                             Los Angeles-San Diego-San Luis Obispo Rail Corridor Agency VAR              82000000.00                    CONST            17000182              R362GA        None                       2016-17                     allocation_ggrf_funding 2016.00                      49995000.00
                                                                                                                                                                                                                                 17000234              R366GA        None                       2016-17                     allocation_ggrf_funding 2016.00                       4017000.00
                                                                                                                                                                                                                                 18000175              R362GB        None                       2016-17                     allocation_ggrf_funding 2016.00                      11988000.00
                                                                                                                                                                                                                                 19000068              R401GB        None                       2017-18                     allocation_ggrf_funding 2016.00                        500000.00
                                                                                                                                                                            